In [1]:
import pandas as pd
import numpy as np
import pyodbc
import openpyxl
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import datetime as dt
import glob
import os
from pathlib import Path
from dateutil.relativedelta import relativedelta

In [3]:
def stpassgn(df):
    if df["store"]=='ARUNDEL':
        return '0011015975'
    elif df["store"]=='FC':
        return '0011016803'
    elif df["store"]=='HM':
        return '0011013949'
    elif df["store"]=='KC':
        return '0011016100'
    elif df["store"]=='MH':
        return '0011011351'
    elif df["store"]=='TAKOMA':
        return '0011014190'
    else :
        return ''

In [4]:
current_date=dt.datetime.today(); current_date
m=1
act=current_date - relativedelta(months=m)
act_date=act.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
#cur_date=act.strftime("%Y-%m-%d %H:%M:%S") Another way to do
# aill_files=Path(r'Y:\OM ONLY_Shared Documents\5 Reports with Power Query\Source Data\POS\RAW\KC\RAW\042022').glob('*042022*.xlsx')
# print(aill_files)
# df=pd.concat([pd.read_excel(f, index_col=None, header=0) for f in aill_files])
# df.info()
prlst = os.listdir(r'Z:\Ivykiss Artwork\ZZ_TEMP\SOM\13_POS\3.RAW\KC\022023')
df_o = pd.DataFrame()
for i in prlst:
   df=pd.read_excel(r'Z:\Ivykiss Artwork\ZZ_TEMP\SOM\13_POS\3.RAW\KC\022023\\' + i, dtype={'UPC': 'str'}, engine='openpyxl')
   df['act_date'] = act_date
   df['temp'] = i[6:]
   df['store'] = df['temp'].str.split('(', 1).str[0]
   df['category'] = df['temp'].str.split('\((.*?)\)', 1).str[1]
   df.drop(columns=['temp'], inplace=True)
   df_o = df_o.append(df) 
df_o['shiptoparty'] = df_o.apply(stpassgn, axis=1)
df_o.columns=df_o.columns.str.lower()
df_o=df_o.rename(columns={"amount":"gross_amt","count":"qty"})
df_o=df_o[["act_date","shiptoparty","upc","gross_amt","qty"]]
df_o['upc'] = pd.to_numeric(df_o['upc'], errors='coerce')
df_o.dropna(subset=["upc"], inplace=True)
df_o['upc']=df_o['upc'].astype(str)
df_o['upc']=df_o['upc'].str.replace("\.0", "")
df_o=df_o[(df_o["gross_amt"]>0) & (df_o["qty"]>0)]

C:\Users\KISSAD~1\AppData\Local\Temp/ipykernel_15500/616074461.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df_o['upc']=df_o['upc'].str.replace("\.0", "")


In [5]:
df_o.head()

,act_date,shiptoparty,upc,gross_amt,qty
0,2023-02-01,0011015975,43917331003,6.02,2
1,2023-02-01,0011015975,54890000336,7.35,7
2,2023-02-01,0011015975,72632894999,33.90,2
3,2023-02-01,0011015975,74108217417,11.65,1
4,2023-02-01,0011015975,91952793928,33.88,4


In [6]:
df_o["shiptoparty"].value_counts()

0011016100    5367
0011015975    4446
0011016803    4075
0011013949    3808
0011014190    3228
0011011351    2141
Name: shiptoparty, dtype: int64

In [7]:
df_o.count()

act_date       23065
shiptoparty    23065
upc            23065
gross_amt      23065
qty            23065
dtype: int64

In [8]:
df_o.tail()

,act_date,shiptoparty,upc,gross_amt,qty
302,2023-02-01,0011014190,888432952704,10.59,1
303,2023-02-01,0011014190,888432952728,8.47,1
304,2023-02-01,0011014190,888432962130,21.15,5
305,2023-02-01,0011014190,888432977585,5.29,1
306,2023-02-01,0011014190,888432984002,5.29,1


In [9]:
df_o.sort_values(by=["gross_amt"], ascending=False)

,act_date,shiptoparty,upc,gross_amt,qty
1795,2023-02-01,0011016100,854102006732,867.51,63
1388,2023-02-01,0011013949,854102006732,589.35,43
779,2023-02-01,0011016100,617218000014,537.03,39
659,2023-02-01,0011016100,77312411039,508.11,37
670,2023-02-01,0011013949,617218000014,507.76,40
...,...,...,...,...,...
143,2023-02-01,0011011351,824703004567,1.00,1
79,2023-02-01,0011016803,81555764005,0.94,1
1046,2023-02-01,0011014190,8853976005027,0.84,1
174,2023-02-01,0011014190,843632033240,0.82,1


In [10]:
df_o.describe()

,gross_amt,qty
count,23065.000000,23065.000000
mean,16.943197,2.955907
std,24.910652,12.649378
min,0.630000,1.000000
25%,5.290000,1.000000
50%,10.470000,2.000000
75%,19.060000,3.000000
max,867.510000,1807.000000


In [11]:
df_o.sort_values(by=["qty"], ascending=False)

,act_date,shiptoparty,upc,gross_amt,qty
7,2023-02-01,0011013949,1000005,90.29,1807
536,2023-02-01,0011016100,843632019497,172.70,165
374,2023-02-01,0011013949,843632021643,219.01,104
341,2023-02-01,0011013949,843632019497,102.67,97
436,2023-02-01,0011015975,843632021643,95.83,92
...,...,...,...,...,...
1482,2023-02-01,0011013949,896981000159,12.71,1
1484,2023-02-01,0011013949,897127000774,7.41,1
0,2023-02-01,0011013949,11509004031,5.29,1
8,2023-02-01,0011013949,73930316404,5.29,1


In [12]:
df_o["alpha"]=list(map(lambda x: x.isalpha(), df_o["upc"]))
df_o["alpha"].unique()

array([False])

In [13]:
df_o=df_o[["act_date","shiptoparty","upc","gross_amt","qty"]]; df_o

,act_date,shiptoparty,upc,gross_amt,qty
0,2023-02-01,0011015975,43917331003,6.02,2
1,2023-02-01,0011015975,54890000336,7.35,7
2,2023-02-01,0011015975,72632894999,33.90,2
3,2023-02-01,0011015975,74108217417,11.65,1
4,2023-02-01,0011015975,91952793928,33.88,4
...,...,...,...,...,...
302,2023-02-01,0011014190,888432952704,10.59,1
303,2023-02-01,0011014190,888432952728,8.47,1
304,2023-02-01,0011014190,888432962130,21.15,5
305,2023-02-01,0011014190,888432977585,5.29,1


In [15]:
server = '10.1.3.25' 
database = '**' 
username = '**' 
password = '**' 
connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
print("connected")

connected


In [16]:
 df_o.to_sql('ivy.sd.fact.pos', engine, schema = "dbo", if_exists='append', index=False, chunksize=1000)
 print("\n Successfully Transported")


 Successfully Transported
